In [2]:
import pandas as pd
import numpy as np
from helper import get_mapper
import json

In [21]:
bpm = pd.read_csv("../basic/block_plant_mapper.csv")

In [5]:
CDF = pd.read_csv("CDF.csv", parse_dates=['produced_at'])

In [6]:
gy = CDF.resample('1Y', on='produced_at').sum()
gm = CDF.resample('1M', on='produced_at').sum()

In [7]:
gm2 = gm.reset_index()
gm2['year'] = gm2['produced_at']
gm2['month'] = gm2['produced_at']

In [8]:
gm2['year'] = gm2['year'].apply(lambda x: str(x).split("-")[0])
gm2['month'] = gm2['month'].apply(lambda x: int(str(x).split("-")[1]))
gm2['month'] = gm2['month'].astype(int)

In [9]:
gm3 = gm2.sort_values(by=["year", 'month'])
gm4 = gm3.drop(columns='produced_at')

In [10]:
gm5 = gm4.melt(id_vars=["year", "month"], var_name='blockid', value_name='power')
gm5['power'] = gm5['power'].astype(int)

In [11]:
gm5

,year,month,blockid,power
0,2015,1,BNA0067,70621
1,2015,2,BNA0067,68088
2,2015,3,BNA0067,66709
3,2015,4,BNA0067,53328
4,2015,5,BNA0067,76938
...,...,...,...,...
14935,2019,8,BNA1093,58135
14936,2019,9,BNA1093,58514
14937,2019,10,BNA1093,110960
14938,2019,11,BNA1093,151811


In [22]:
pm1 = gm5.merge(bpm, on="blockid")

In [14]:
gm6 = gm5.copy()

In [12]:
gm6 = gm5.groupby(by=['blockid', 'month', 'year'])

In [25]:
m = pm1.duplicated(subset=['year', 'month', 'plantid'])

In [26]:
pm1.loc[m]

,year,month,blockid,power,plantid
120,2015,1,BNA0124,532722,14-70-46630660001
121,2015,2,BNA0124,496284,14-70-46630660001
122,2015,3,BNA0124,507962,14-70-46630660001
123,2015,4,BNA0124,541273,14-70-46630660001
124,2015,5,BNA0124,576302,14-70-46630660001
...,...,...,...,...,...
12955,2019,8,BNA1093,58135,06-09-100-0003-0003
12956,2019,9,BNA1093,58514,06-09-100-0003-0003
12957,2019,10,BNA1093,110960,06-09-100-0003-0003
12958,2019,11,BNA1093,151811,06-09-100-0003-0003


In [35]:
pm2 = pm1.copy()

In [36]:
pm2['plantpower'] = pm2.groupby(['plantid', 'month', 'year'])['power'].transform('sum')

In [37]:
pm3 = pm2.drop_duplicates(['month', 'year', 'plantid'])

In [38]:
pm3.sort_values(by="plantpower", ascending=False)

,year,month,blockid,power,plantid,plantpower
9343,2018,8,BNA1401b,641439,06-05-100-0248923,2633329
9321,2016,10,BNA1401b,723383,06-05-100-0248923,2631880
9345,2018,10,BNA1401b,688779,06-05-100-0248923,2608109
9342,2018,7,BNA1401b,609273,06-05-100-0248923,2577498
9338,2018,3,BNA1401b,702196,06-05-100-0248923,2557329
...,...,...,...,...,...,...
10486,2018,11,BNA0936,0,06-08-1195689,0
10487,2018,12,BNA0936,0,06-08-1195689,0
10488,2019,1,BNA0936,0,06-08-1195689,0
10489,2019,2,BNA0936,0,06-08-1195689,0


In [40]:
pm4 = pm3.drop(['blockid', 'power'], axis=1)

In [41]:
pm4

,year,month,plantid,plantpower
0,2015,1,06-05-900-0205103,70621
1,2015,2,06-05-900-0205103,68088
2,2015,3,06-05-900-0205103,66709
3,2015,4,06-05-900-0205103,53328
4,2015,5,06-05-900-0205103,76938
...,...,...,...,...
12895,2019,8,06-09-100-0003-0003,58135
12896,2019,9,06-09-100-0003-0003,58514
12897,2019,10,06-09-100-0003-0003,110960
12898,2019,11,06-09-100-0003-0003,151811


In [43]:
pm4.to_csv("monthlyp_pg.csv", header=False)